In [ ]:
%pylab inline
#!/usr/bin/env python -u
# -*- coding: utf-8 -*-

import glob
import os
import re
import sys

import cv2
import numpy as np

## Step 1. Extract video frames

e.g. every 10th frame:

```mkdir frames
ffmpeg -i VIDEO.mp4 -vf "setpts=0.1*PTS" -q 2 frames\%04d.jpg```

In [ ]:
# working_dir = os.getcwd()
# if len(sys.argv) == 2:
#     working_dir = sys.argv[1]
# elif len(sys.argv) > 2:
#     print "Usage: %s [path]" % os.path.basename(sys.argv[0])
#     sys.exit(-1)

working_dir = r"calibration\frames"

# Global settings (checkerboard).
# pattern_size = (8, 6)
# square_size = 24  # mm

# Global settings (asymmetric circles).
pattern = 'acircles'
pattern_size = (4, 11)
square_size = 20  # mm

In [ ]:
# 3D world positions of checkerboard corners / circle centres.
pattern_points = []

if pattern == 'checkerboard' or pattern == 'circles':  # circles UNTESTED (copied from OpenCV calibration.cpp sample)
    for y in range(pattern_size[1] - 1, -1, -1):
        for x in range(0, pattern_size[0]):
            pattern_points.append([x * square_size, y * square_size, 0])

elif pattern == 'acircles':
    for y in range(pattern_size[1] - 1, -1, -1):
        for x in range(0, pattern_size[0]):
            pattern_points.append([(2 * x + y % 2) * square_size, y * square_size, 0])

pattern_points = np.array(pattern_points, np.float32)

In [ ]:
# All files in the directory.
filenames = glob.glob(os.path.join(working_dir, "*.*"))

# Keep only images with numeric filename.
image_pattern = re.compile("^\d+\.(?:jpe?g|png|bmp|tiff?)$", re.I)
filenames = [e for e in filenames if re.findall(image_pattern, os.path.basename(e))]

filenames.sort()
print("{0} images found.".format(len(filenames)))

In [ ]:
# Use every tenth image.
filenames = filenames[::10]

In [ ]:
# Pick the n sharpest images from a subset of all images.
# (This cell can be skipped to process all images, e.g. if they were manually selected.)

# First compute sharpness of every image using variance of Laplacian.
# See 'LAPV' case of Matlab code in <http://stackoverflow.com/a/7768918/>.
sharpness = []
for filename in filenames:
    image = cv2.imread(filename, 0)
    sharpness.append((filename, cv2.Laplacian(image, cv2.CV_64F).var()))

# Sort by sharpness (highest first)
sharpness.sort(key=lambda x: x[1], reverse=True)
# plot(np.array([e[1] for e in blurriness]))
# for filename, blur in blurriness[:20]:
#     image = cv2.imread(filename, 0)
#     output_filename = "%sblurry-%s-%06.2f.jpg" % (filename[:-8], filename[-8:-4], blur)
#     cv2.imwrite(output_filename, image)

# Keep only the sharpest n images.
filenames = [filename for filename, sharpness in sharpness[:20]]
filenames.sort()

In [ ]:
# Increase the maximum area of the blob detector to also detect really big circles.
params = cv2.SimpleBlobDetector_Params()
params.maxArea = 150000
detector = cv2.SimpleBlobDetector_create(params)

# # # Detect blobs.
# img = cv2.imread(r'calibration\0101.jpg', 0)
# keypoints = detector.detect(img)

# im_with_keypoints = cv2.drawKeypoints(img, keypoints, np.array([]), (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# # imshow(im_with_keypoints)
# cv2.imwrite(r'calibration\0101-features.jpg', im_with_keypoints)

In [ ]:
world_points = []
image_points = []
filenames_used = []

for filename in filenames:
    if filename[-3:].lower() not in ['jpg', 'png', 'bmp', 'tif']:
        continue
    print("Processing image '%s' ... " % filename, end='')

    ## Read points from cache file if it exists.
    cacheFilename = "{filename}-{pattern}-points.npz".format(**locals())
    if os.path.exists(cacheFilename):
        with open(cacheFilename, 'rb') as fh:
            cacheFile = np.load(fh)
            image_size = (cacheFile['image_size'][1], cacheFile['image_size'][0])
            points = cacheFile['points']
            print("Cached.")
            
    else:  ## Compute points from scratch and cache them
        image = cv2.imread(filename)
        image_size = image.shape[:2]

        if pattern == 'checkerboard':
            (retval, points) = cv2.findChessboardCorners(image, pattern_size)
        elif pattern == 'circles':  # UNTESTED
            (retval, points) = cv2.findCirclesGrid(image, pattern_size, flags=cv2.CALIB_CB_SYMMETRIC_GRID | cv2.CALIB_CB_CLUSTERING, blobDetector=detector)
        elif pattern == 'acircles':
            (retval, points) = cv2.findCirclesGrid(image, pattern_size, flags=cv2.CALIB_CB_ASYMMETRIC_GRID)

        if not retval:
            print("Points not found.")
            continue

        # Write images with detected points
        cv2.drawChessboardCorners(image, pattern_size, points, retval)
        cv2.imwrite(filename + "-points.jpeg", image)  # using '.jpeg' so that these images are never used as inputs

        ## Cache point positions.
        with open(cacheFilename, 'wb') as fh:
            np.savez(fh, image_size=image_size, points=points)
            image_size = (image_size[1], image_size[0])

        print("Done.")

    filenames_used.append(filename)
    world_points.append(pattern_points)
    image_points.append(points.reshape(-1, 2))

In [ ]:
len(filenames_used)

In [ ]:
# Initial guess of calibration.
# camera_matrix = np.matrix([[2 * 960., 0., image_size[0] / 2.], [0., 2 * 960., image_size[1] / 2.], [0., 0., 1.]])
# dist_coefs = np.array([-0.27, 0.058, 0., 0., 0.])
camera_matrix, dist_coefs = None, None

# Calibration flags.
calibration_flags = (cv2.CALIB_USE_INTRINSIC_GUESS if camera_matrix is not None and dist_coefs is not None else 0)
calibration_flags |= cv2.CALIB_FIX_ASPECT_RATIO
# calibration_flags |= cv2.CALIB_RATIONAL_MODEL
# calibration_flags |= cv2.CALIB_FIX_PRINCIPAL_POINT
calibration_flags |= cv2.CALIB_ZERO_TANGENT_DIST

# Intrinsic calibration.
(rms, camera_matrix, dist_coefs, rvecs, tvecs) = cv2.calibrateCamera(world_points, image_points, image_size,
    camera_matrix, dist_coefs, flags=calibration_flags
    # | cv2.CALIB_FIX_FOCAL_LENGTH | cv2.CALIB_FIX_K1 | cv2.CALIB_FIX_K2 | cv2.CALIB_FIX_K3 | cv2.CALIB_FIX_K4 | cv2.CALIB_FIX_K5 | cv2.CALIB_FIX_ASPECT_RATIO,
)

print()
# print "K  =", np.array_str(camera_matrix, precision=6)
print("K  =", camera_matrix)
# print "kc =", np.array_str(dist_coefs.ravel(), precision=6)
print("kc =", dist_coefs)
print()
print("Reprojection error: %.6f" % rms)

with open(os.path.join(working_dir, 'intrinsics.txt'), 'w') as fh:
    fh.write("KMatrix = %s\n"    % ' '.join(str(e) for e in camera_matrix.ravel()))
    fh.write("DistCoeffs = %s\n" % ' '.join(str(e) for e in dist_coefs.ravel()))
    fh.write("ImageSize = %s\n" % ' '.join(str(e) for e in image_size))
    fh.write("ReprojError = %f\n" % rms)

----
## Undistortion

In [ ]:
# Read in camera intrinsics.
with open(os.path.join(working_dir, 'intrinsics.txt'), 'r') as fh:
    for line in fh:
        line = line.strip()
        var, value = [e.strip() for e in line.split('=')]
        if var == 'KMatrix': camera_matrix = np.array([float(e) for e in value.split()]).reshape([3,3])
        if var == 'DistCoeffs': dist_coefs = np.array([float(e) for e in value.split()])
        if var == 'ImageSize': image_size = tuple([int(e) for e in value.split()])
        if var == 'ReprojError': rms = float(value)

print("Read calibration data:")
print("  KMatrix = %s"    % str(camera_matrix))
print("  DistCoeffs = %s" % str(dist_coefs))
print("  ImageSize = %s" % str(image_size))
print("  ReprojError = %f" % rms)

In [ ]:
# image_size = (1080, 1920)

# new_camera_matrix = np.matrix(camera_matrix)
# new_camera_matrix = np.matrix([[camera_matrix[0, 0], 0, image_size[0] / 2.], [0, camera_matrix[1, 1], image_size[1] / 2.], [0, 0, 1]])
# new_camera_matrix = np.matrix([[1900, 0, image_size[0] / 2.], [0, 1900, image_size[1] / 2.], [0, 0, 1]])  # iPhone X
# new_camera_matrix = np.matrix([[1100, 0, image_size[0] / 2.], [0, 1100, image_size[1] / 2.], [0, 0, 1]])  # Huawei P9 lite
new_camera_matrix = np.matrix([[1100, 0, image_size[0] / 2.], [0, 1100, image_size[1] / 2.], [0, 0, 1]])  # GoPro 2028p
# new_camera_matrix = np.matrix([[800, 0, image_size[0] / 2.], [0, 800, image_size[1] / 2.], [0, 0, 1]])  # GoPro 1440p
print(new_camera_matrix)

In [ ]:
# Projecting world points into images for verification.
for index, filename in enumerate(filenames_used):
    print("Visualising image '%s' ... " % filename, end='')

    proj_points = new_camera_matrix * (np.mat(cv2.Rodrigues(rvecs[index])[0]) * np.mat(world_points[index].T) + tvecs[index])
    proj_points = np.divide(proj_points[:2,:], proj_points[2,:])
    
    img = cv2.imread(filename)
    img = cv2.undistort(img, camera_matrix, dist_coefs, newCameraMatrix=new_camera_matrix)
    cv2.imwrite(filename + "-undistorted.jpeg", img)
    cv2.drawChessboardCorners(img, pattern_size, np.array(proj_points.T, np.float32)[:,np.newaxis,:], True)
    cv2.imwrite(filename + "-points-projected.jpeg", img)

    print("Done.")

In [ ]:
# Undistort all frames.
input_dir = working_dir
output_dir = os.path.join(os.path.dirname(input_dir), "undistorted")
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
# with file(os.path.join(output_dir, 'calibration.txt'), 'w') as fh:
#     fh.write("undistorted images with K =\n{new_camera_matrix}".format(**locals()))
with open(os.path.join(output_dir, 'instrinsics.txt'), 'w') as fh:
    fh.write("KMatrix = %s\n"    % ' '.join(str(e) for e in np.array(new_camera_matrix).ravel()))
    fh.write("DistCoeffs = 0 0 0 0 0\n")

for filename in glob.glob(os.path.join(input_dir, "*.jpg")):
    print("Undistorting image '%s' ... " % filename, end='')
    img = cv2.imread(filename)
    img = cv2.undistort(img, camera_matrix, dist_coefs, newCameraMatrix=new_camera_matrix)
    cv2.imwrite(os.path.join(output_dir, 'undistorted-' + os.path.basename(filename)), img)
    print("Done.")